In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns



# ks2018 = pd.read_csv('data/ks-projects-201801.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')


In [ ]:
train.head()

In [ ]:
train.describe()


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [18]:
def find_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [19]:
find_data(train)


,Total,Percent
usd pledged,3797,1.002744
name,4,0.001056
usd_goal_real,0,0.000000
usd_pledged_real,0,0.000000
country,0,0.000000
backers,0,0.000000
state,0,0.000000
pledged,0,0.000000
launched,0,0.000000
goal,0,0.000000


In [20]:
train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

name             375764
category            159
main_category        15
currency             14
deadline           3164
launched         378089
state                 6
country              23
dtype: int64

In [24]:
# trainInfo = train['ID'].value_counts()


In [26]:
# df = pd.DataFrame({'labels': ksInfo.index,
#                    'values':ksInfo.values
#                   })

In [ ]:
# plt.figure(figsize = (6,6))
# plt.title('Loans rep')
# sns.set_color_codes("colorblind")
# sns.barplot(x = 'labels', y="values", data=df)
# locs, labels = plt.xticks()
# plt.show()